<a href="https://colab.research.google.com/github/pattrickx/wordle_breaker/blob/main/Termo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [348]:
!pip3 install unidecode

In [349]:
import pandas as pd
import unidecode 
df = pd.read_csv("/content/drive/MyDrive/My_projects/teste/palavras.txt")

In [350]:
df

,palavras
0,ababa
1,ababá
2,abacá
3,abaci
4,ábaco
...,...
3965,zuate
3966,zumbi
3967,zunir
3968,zurro


In [351]:
df["palavras"] = df["palavras"].apply(lambda r:unidecode.unidecode(r))
df = df[df["palavras"].apply(lambda r:len(r)==5)]
df.drop_duplicates(inplace=True)
df

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,palavras
0,ababa
2,abaca
3,abaci
4,abaco
5,abada
...,...
3965,zuate
3966,zumbi
3967,zunir
3968,zurro


In [352]:
letras_certas = [False, False, "i", False, False]
letras_erradas = ["a","n","d","p","c"]
letras_existentes = [{"letra":"e","posicao":[False, True, False, True, False]},
                     {"letra":"s","posicao":[True, True, False, True, False]}
                     ]

In [353]:
def finde_letra(palavra,letras):
    for l in letras:
        if l in palavra:
            return True
    return False

In [354]:
def finde_letras_existentes(p,letras):
    cont = 0
    for l in letras:
        for idx in range(5):
            if l["posicao"][idx]:
                if l["letra"]==p[idx]:
                    cont+=1
                    break
            else:
                if l["letra"]==p[idx]:
                    return False
               
    if cont == len(letras):
        return True
    return False

In [355]:
def find_letras_certas(p,letras_certas):
    for i,letra in enumerate(letras_certas):
        if letra != False:
            if not letra==p[i]:
                return False
    return True

In [356]:
filto_erradas = df.apply(lambda row: not finde_letra(row["palavras"],letras_erradas), axis=1)

In [357]:
palavras_sem_letras_erradas = df[filto_erradas]

In [358]:
filtro_letras_existentes = palavras_sem_letras_erradas.apply(lambda row: finde_letras_existentes(row["palavras"],letras_existentes), axis=1)

In [359]:
palavras_com_letras_existentes = palavras_sem_letras_erradas[filtro_letras_existentes]

In [360]:
filtro_letras_certas = palavras_com_letras_existentes.apply(lambda row: find_letras_certas(row["palavras"],letras_certas), axis=1)

In [361]:
palavras_com_letras_certas = palavras_com_letras_existentes[filtro_letras_certas]

In [362]:
palavras_com_letras_certas

,palavras
3221,seixo
